## Appendix Lavaan Syntax

### **Running the Analysis Manually with R Code**

In the previous sections, we explored the results of our causal analysis using the interactive `EffectLiteR` Shiny web application. Now, we will demonstrate how to perform a similar analysis manually by writing and executing R code directly.

Because our model is complex and includes several latent variables , we will use a robust **two-step approach** to ensure the model runs correctly and to simplify the interpretation.

1.  **First, we will fit a measurement model.** We will use the `lavaan` package, which `EffectLiteR` is built on, to define the relationships between our observed survey items and their underlying latent constructs. From this model, we will calculate the "factor scores" for each latent variable, effectively turning our unobserved constructs into new, calculated variables for each participant.

2.  **Second, we will run the main `effectLite` analysis.** However, instead of a complex latent variable model, we will use a simpler model that uses the factor scores we just created as regular, observed covariates. This simplifies the process while still accounting for the nuanced concepts measured by the latent variables.

---
> **Important Note on the Model**
>
> Please note that the two-step approach shown in the code below represents a slightly simplified version of the full model. The `EffectLiteR` Shiny app, by default, runs a more complex, fully simultaneous Structural Equation Model.
>
> You may notice some small differences between the results from this script and the results from the Shiny app. These differences arise from the different modeling strategies (two-step vs. fully simultaneous). The complete `lavaan` syntax for the model generated by the Shiny app can be found under the **"lavaan Syntax"** tab within that application. The explanation of the syntax is also provided below, including for the output. As a homework, please compare this two-step vs. fully simultaneous approaches.
---

In [ ]:
#######################################################
## Specify R environment for rpy2
#import os
#os.environ['R_HOME'] = r'C:/Program Files/R/R-4.4.3'  # Replace with your R path
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
import contextlib
# Ipython extension for plotting
%load_ext rpy2.ipython
########################################################

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display


# Read the CSV file using a relative path
effects = pd.read_csv("../EffectLiteR/Datasets/sophonet_data_simulated.csv")
effects.head()


# Activate pandas2ri for automatic conversion
pandas2ri.activate()

# Install the effectliter package in R
#ro.r('install.packages("EffectLiteR")')

# Import the package
ro.r('library(EffectLiteR)')

# Load the dataset into R
ro.globalenv['effects'] = pandas2ri.py2rpy(effects)

c:\Users\danie\.conda\envs\psy112ER\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(
R[write to console]: Installing package into 'C:/Users/danie/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



--- Please select a CRAN mirror for use in this session ---


R[write to console]: trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.4/EffectLiteR_0.5-1.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 567023 bytes (553 KB)

R[write to console]: downloaded 553 KB




package 'EffectLiteR' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\danie\AppData\Local\Temp\RtmpqoyTVu\downloaded_packages


R[write to console]: Loading required package: lavaan

R[write to console]: This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.



In [2]:
# --- STEP 1: FIT MEASUREMENT MODEL AND COMPUTE LATENT VARIABLE SCORES (Corrected) ---
print("\n--- STEP 1: Computing Latent Variable Scores using lavaan ---")

# Assign the pandas dataframe to the R environment
ro.r.assign("effects_data", effects)

# Define the R code for this step in a multiline string
step1_r_code = """
library(lavaan)

# Define ONLY the measurement model to create the latent variables.
measurement_syntax <- "
  eta =~ lsas.a.t2 + lsas.v.t2
  xi1 =~ lsas.a.t1 + lsas.v.t1
  xi2 =~ bdi.t1.i1 + bdi.t1.i2 + bdi.t1.i3
  xi3 =~ ecr.anx.t1.i1 + ecr.anx.t1.i2 + ecr.anx.t1.i3
  xi4 =~ ecr.avoi.t1.i1 + ecr.avoi.t1.i2 + ecr.avoi.t1.i3
  xi5 =~ tpq.ha.i1 + tpq.ha.i2 + tpq.ha.i3
"

# Fit a Confirmatory Factor Analysis (CFA) model
fit <- try(cfa(measurement_syntax, data = effects_data, missing = "listwise"), silent = TRUE)

# Predict the factor scores for the latent variables using the correct type
latent_scores <- as.data.frame(lavPredict(fit, type = "lv"))

# Combine the original data with the new latent variable scores
effects_with_scores <- cbind(effects_data, latent_scores)
"""
# Execute the R code for Step 1
ro.r(step1_r_code)


# ==============================================================================
# STEP 2: RUN EFFECTLITER USING ONLY MANIFEST VARIABLES
# ==============================================================================
print("\n--- STEP 2: Running EffectLiteR with Computed Scores ---")

# Define the R code for the simplified EffectLiteR call
step2_r_code = """
library(EffectLiteR)

# Define the outcome, treatment, and ALL covariates 
# The covariates now include the original manifest ones AND the new latent scores
y_var <- "eta"
x_var <- "tb"
z_vars <- c("comorbid", "iip.lov", "iip.dom", 
            "xi1", "xi2", "xi3", "xi4", "xi5")

# Run the simplified EffectLiteR model. 
# Note we no longer need the 'k', 'measurement', or 'syntax' arguments.
elr_model <- effectLite(
    data = effects_with_scores,
    y = y_var,
    x = x_var,
    z = z_vars,
    missing = "fiml",
    estimator = "mlr",
    homoscedasticity = FALSE
)
"""
# Execute the R code for Step 2
ro.r(step2_r_code)


# ==============================================================================
# STEP 3: PRINT ALL RESULTS (This section should now work)
# ==============================================================================
print("\n" + "=" * 50)
print("EFFECTLITER ANALYSIS RESULTS")
print("=" * 50)
main_results = ro.r('elr_model')
print(main_results)



--- STEP 1: Computing Latent Variable Scores using lavaan ---

--- STEP 2: Running EffectLiteR with Computed Scores ---

EFFECTLITER ANALYSIS RESULTS


--------------------- Message  --------------------- 

 -- model converged succesfully -- 


--------------------- Variables  --------------------- 

Outcome variable Y:  eta 
Treatment variable X:  tb   (Reference group:  CBT)
Continuous covariates in Z=(Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8): Z1=comorbid Z2=iip.lov Z3=iip.dom Z4=xi1 Z5=xi2 Z6=xi3 Z7=xi4 Z8=xi5 

Levels of Treatment Variable X 
   X   tb (original)   Indicator
   0             CBT       I_X=0
   1            STPP       I_X=1


 --------------------- Regression Model --------------------- 

 E(Y|X,Z) = g0(Z) + g1(Z)*I_X=1 
  g0(Z) = g000 + g001 * Z1 + g002 * Z2 + g003 * Z3 + g004 * Z4 + g005 * Z5 + 
            + g006 * Z6 + g007 * Z7 + g008 * Z8
  g1(Z) = g100 + g101 * Z1 + g102 * Z2 + g103 * Z3 + g104 * Z4 + g105 * Z5 + 
            + g106 * Z6 + g107 * Z7 + g108 * Z8

 Intercept 

### Explanation of Lavaan Syntax from EffectLiteR Shiny App

In [ ]:
#### lavaan Syntax for EffectLiteR Model ####

# ==============================================================================
# SECTION 1: Measurement Model
# The purpose of this section is to define the latent variables (unobserved
# constructs) from the observed indicator variables (the actual survey items).
# ==============================================================================

## Define the latent outcome variable 'eta' (social phobia at time 2)
eta =~ c(1,1)*lsas.a.t2      # Fixes the loading of lsas.a.t2 to 1 in both groups for identification.
eta =~ c(1,1)*lsas.v.t2      # Fixes the loading of lsas.v.t2 to 1 in both groups. 'eta' is measured by fear (lsas.a) and avoidance (lsas.v).

## Define the latent covariates (Xi variables) from their indicators at time 1
xi1 =~ c(1,1)*lsas.a.t1      # Defines 'xi1' (baseline social phobia) from the fear subscale.
xi1 =~ c(1,1)*lsas.v.t1      # 'xi1' is also measured by the avoidance subscale.
xi2 =~ c(1,1)*bdi.t1.i1      # Defines 'xi2' (likely depression) using the first BDI indicator.
xi2 =~ c(la123,la123)*bdi.t1.i2 # Freely estimates the loading for the 2nd BDI indicator, but constrains it to be equal across groups (measurement invariance).
xi2 =~ c(la133,la133)*bdi.t1.i3 # Constrains the loading for the 3rd BDI indicator to be equal across groups.
xi3 =~ c(1,1)*ecr.anx.t1.i1   # Defines 'xi3' (likely attachment anxiety).
xi3 =~ c(la124,la124)*ecr.anx.t1.i2 # ...and so on for the remaining latent variables and their indicators.
xi3 =~ c(la134,la134)*ecr.anx.t1.i3
xi4 =~ c(1,1)*ecr.avoi.t1.i1
xi4 =~ c(la125,la125)*ecr.avoi.t1.i2
xi4 =~ c(la135,la135)*ecr.avoi.t1.i3
xi5 =~ c(1,1)*tpq.ha.i1
xi5 =~ c(la126,la126)*tpq.ha.i2
xi5 =~ c(la136,la136)*tpq.ha.i3

## Define the intercepts of the observed indicator variables
# This is necessary for identifying the means of the latent variables.
lsas.a.t2 ~ c(0,0)*1         # Fixes the intercept of this indicator to 0 in both groups.
lsas.v.t2 ~ c(la021,la021)*1   # Freely estimates the intercept but constrains it to be equal across groups.
# ...and so on for the remaining indicators. One indicator per latent variable has its intercept fixed to 0.
lsas.a.t1 ~ c(0,0)*1
lsas.v.t1 ~ c(la022,la022)*1
bdi.t1.i1 ~ c(0,0)*1
bdi.t1.i2 ~ c(la023,la023)*1
bdi.t1.i3 ~ c(la033,la033)*1
ecr.anx.t1.i1 ~ c(0,0)*1
ecr.anx.t1.i2 ~ c(la024,la024)*1
ecr.anx.t1.i3 ~ c(la034,la034)*1
ecr.avoi.t1.i1 ~ c(0,0)*1
ecr.avoi.t1.i2 ~ c(la025,la025)*1
ecr.avoi.t1.i3 ~ c(la035,la035)*1
tpq.ha.i1 ~ c(0,0)*1
tpq.ha.i2 ~ c(la026,la026)*1
tpq.ha.i3 ~ c(la036,la036)*1

# ==============================================================================
# SECTION 2: Structural Model
# The purpose of this section is to define the causal relationships between the
# latent outcome, the treatment, and the covariates.
# ==============================================================================

## Define the regression of the outcome 'eta' on the treatment and covariates
# This is a multigroup model where effects can differ between group 0 (CBT) and group 1 (STPP).
eta ~ c(a000,a100)*1         # Estimates separate intercepts for each group (a000 for group 0, a100 for group 1).
eta ~ c(a001,a101)*comorbid  # Estimates separate regression slopes (effects) of 'comorbid' for each group.
eta ~ c(a002,a102)*iip.lov    # Estimates separate slopes for 'iip.lov'.
eta ~ c(a003,a103)*iip.dom    # ...and so on for all covariates.
eta ~ c(a004,a104)*xi1
eta ~ c(a005,a105)*xi2
eta ~ c(a006,a106)*xi3
eta ~ c(a007,a107)*xi4
eta ~ c(a008,a108)*xi5

## Model the means of the covariates, allowing them to differ by group
comorbid ~ c(mz001,mz101)*1 # Estimates the mean of 'comorbid' for group 0 (mz001) and group 1 (mz101).
iip.lov ~ c(mz002,mz102)*1  # ...and so on for all covariates. This is important for calculating adjusted means later.
iip.dom ~ c(mz003,mz103)*1
xi1 ~ c(mz004,mz104)*1
xi2 ~ c(mz005,mz105)*1
xi3 ~ c(mz006,mz106)*1
xi4 ~ c(mz007,mz107)*1
xi5 ~ c(mz008,mz108)*1

## Allow all covariates to be correlated with each other
# The '~~' operator specifies a covariance.
comorbid ~~ iip.lov          # This line allows 'comorbid' and 'iip.lov' to be correlated.
# ...this is repeated for all possible pairs of covariates.
comorbid ~~ iip.dom
# ... (many covariance lines omitted for brevity) ...
xi4 ~~ xi5

# ==============================================================================
# SECTION 3: User-Defined Parameters
# The purpose of this section is to use the estimated 'a' and 'mz' parameters
# from the model to calculate the specific quantities of interest, like the
# 'g' coefficients, average effects, and adjusted means.
# The ':=' operator is used to define a new parameter.
# ==============================================================================

## Relative Group Frequencies
# This section models the proportion of participants in each group.
group % c(gw0,gw1)*w
N := exp(gw0) + exp(gw1)       # Total size based on log-linear modeling.
relfreq0 := exp(gw0)/N         # Relative frequency (proportion) of the control group (X=0).
relfreq1 := exp(gw1)/N         # Relative frequency (proportion) of the treatment group (X=1).

## beta Coefficients
# This is an intermediate step to simply relabel the 'a' coefficients to 'b' for clarity.
b000 := a000
# ... (all 'a' parameters are relabeled to 'b' parameters) ...
b108 := a108

## gamma Coefficients
# This section calculates the 'g' parameters for the EffectLiteR framework.
g000 := b000                 # The intercept for the control group's outcome (g0 function).
g001 := b001                 # The effect of covariate 1 in the control group.
# ...and so on for the g0 function.
g100 := b100-b000            # The main treatment effect (difference in intercepts). This is the intercept of the g1 function.
g101 := b101-b001            # The interaction effect for covariate 1 (difference in slopes).
# ...and so on. This re-parameterizes the model to be about a baseline (g0) and an effect (g1).

## Unconditional Expectations E(Z)
# Calculates the overall average value for each covariate across the whole sample.
Ez1 := mz001*relfreq0 + mz101*relfreq1 # The overall mean of the first covariate.
# ...calculated for all covariates.

## Average Effects
# Calculates the Average Causal Effect (ACE) by plugging the overall average covariate
# values (Ez) into the effect function g1(Z).
Eg1 := g100*1 + g101*Ez1 + g102*Ez2 + g103*Ez3 + g104*Ez4 + g105*Ez5 + g106*Ez6 + g107*Ez7 + g108*Ez8

## Adjusted Means
# Calculates the expected outcome for each group, adjusted for the covariates.
adjmean0 := g000*1 + g001*Ez1 + ... # The mean for the control group, calculated at the average value of all covariates.
adjmean1 := adjmean0 + Eg1          # The mean for the treatment group is simply the control group's mean plus the average effect.

## Conditional Expectations E(Z|X=x)
# Calculates the average value of each covariate within each group separately.
Ez1gx0 := mz001 # The mean of covariate 1 for the control group (group X=0).
Ez1gx1 := mz101 # The mean of covariate 1 for the treatment group (group X=1).
# ...calculated for all covariates.

## Effects given X=x
# Calculates the Average effect on the Treated (ATT) and Untreated (ATU).
Eg1gx0 := g100*1 + g101*Ez1gx0 + ... # ATU: Plugs the control group's average covariate values into the effect function.
Eg1gx1 := g100*1 + g101*Ez1gx1 + ... # ATT: Plugs the treatment group's average covariate values into the effect function.

### Interpretation of the `lavaan` Model Output

This output provides a comprehensive look at the results of the structural equation model estimated by `EffectLiteR`. We can interpret it section by section.

---

### Model Fit and Header Information

This first block tells us about the model's convergence and overall fit to the data.

* **Convergence and Observations**:
    * `lavaan (0.5-22) converged normally after 338 iterations`: This is good news. It confirms that the statistical algorithm successfully found a stable solution for the model parameters.
    * `Number of observations per group`: We have **170** participants in Group 0 (CBT) and **158** in Group 1 (STPP).

* **Overall Model Fit Statistics**:
    * `P-value (Chi-square) = 1.000`: The null hypothesis of the Chi-Square test is that the model fits the data perfectly. A high p-value (>.05) means we do not reject this hypothesis. A value of 1.000 indicates a perfect fit.
    * `CFI = 1.000` and `TLI = 1.086`: The Comparative Fit Index (CFI) and Tucker-Lewis Index (TLI) are measures of how well our model fits compared to a baseline "worst-fit" model. Values > 0.95 are considered excellent. The values here indicate a perfect fit.
    * `RMSEA = 0.000` and `SRMR = 0.000`: The Root Mean Square Error of Approximation (RMSEA) and Standardized Root Mean Square Residual (SRMR) are measures of error. Values < 0.06 are considered good. The values of 0.000 here indicate a perfect fit.

> **Overall Conclusion on Fit**: All fit indices suggest that the specified model fits the observed data exceptionally well.

---

### Parameter Estimates (Group-Specific Results)

This section shows the detailed results for each group separately.

#### **Group 1 [0]: The CBT Group**

This block shows the model parameters as they apply to the reference group (CBT).

* **Latent Variables**: This section confirms how the latent constructs (e.g., `xi2`, `xi3`) were measured. The significant `z-value` and `P(>|z|)` columns for the estimated loadings (e.g., `l123`, `l133`) show that the chosen indicator variables are strongly related to their respective latent variables.
* **Regressions**: This shows how the covariates predict the outcome (`eta`) **within the CBT group**. For example:
    * The effect of `comorbd` (`a001`) is **0.119** (p=0.002), meaning higher comorbidity is associated with higher social phobia scores in the CBT group.
    * The effect of `iip.lov` (`a002`) is **-0.315** (p<0.001), meaning this covariate is significantly and negatively associated with social phobia in the CBT group.
* **Intercepts**: This shows the baseline values for all variables in the CBT group. The intercept for the outcome `eta` (`a000`) is **-0.077**, which is the expected social phobia score for someone in the CBT group when all covariates are zero.
* **Variances**: This shows the variance of each variable. The variance for `.eta` (**0.124**) is the *residual* or unexplained variance in the outcome for the CBT group after accounting for all the predictors.

#### **Group 2 [1]: The STPP Group**

This block shows the same parameters, but for the treatment group (STPP).

* **Latent Variables**: The factor loadings (e.g., `l123`) have the exact same estimates as in the CBT group. This is because we constrained them to be equal to test for measurement invariance.
* **Regressions**: This shows how the covariates predict the outcome **within the STPP group**. We can compare these to the CBT group to see where interactions might arise. For example:
    * The effect of `xi1` (`a104`) is **0.778** (p<0.001). This is much larger than the effect in the CBT group (0.414), suggesting an interaction.
    * The effect of `comorbd` (`a101`) is **0.074** (p=0.059), which is smaller than in the CBT group and not statistically significant.

---

### Defined Parameters (The `EffectLiteR` Results)

This final, crucial section calculates the user-friendly causal effects from the model parameters.

* **`g` coefficients**: These show the re-parameterized model of a baseline (`g0`) and an effect (`g1`).
    * `g100` is the main effect, the difference in intercepts: **-0.144**.
    * `g102` is the interaction effect for `iip.lov`: **0.367** (p=0.004). This is a **significant interaction**, meaning the effect of STPP vs. CBT is different for people with different levels of `iip.lov`.
    * `g104` is the interaction effect for `xi1`: **0.364** (p=0.012). This is also a **significant interaction**.

* **`Eg1` (Average Effects)**: This is the Average Causal Effect (ACE).
    * The overall average effect of STPP compared to CBT is **0.176**.
    * This effect is highly statistically significant (p < 0.001).

* **`adjmean` (Adjusted Means)**: These are the covariate-adjusted average outcomes for each group.
    * The adjusted mean for the CBT group (`adjmean0`) is **0.967**.
    * The adjusted mean for the STPP group (`adjmean1`) is **1.143**.
    * (Note: 1.143 - 0.967 = 0.176, which is the average effect).

* **`Eg1gx0` and `Eg1gx1` (Effects given a Treatment Condition)**:
    * `Eg1gx0` (ATU): The effect for those in the CBT group is **0.172** (p<0.001).
    * `Eg1gx1` (ATT): The effect for those in the STPP group is **0.180** (p<0.001).
    * As before, the similarity of these two values suggests the treatment effect is consistent and generalizable across the groups.